# Modelling sequences



When we go from sequences of word embeddings to a document-wise vector representation that can be classified, we have to somehow summarize a sequence of vectors into a single vector. So far, what we have been doing is:

1. Get one embedding per word ($\mathbb{t \times n}$, where $t$ is the sequence length and $n$ is the embedding dimension),
1. Calculate the timewise mean of the words ($\mathbb{1 \times n}$)
1. Proceed to classification with our Residual MLP modules

This is something like this:

<img src="figs/classifier.png" />

The problem with this idea is that the calculation of the mean totally disregards the order of the words - essentially, we are doing a glorified bag-of-words modelling, which seems non-ideal. When we do so, we are We could find some other way to summarize our sequence of words so that we somehow account for the order of words.



# Multi-Head Attention Encoders

If you were not living under a rock in the last few years, you probably heard about something called a "transformer". This is a neural network topology made famous in an article called [Attention is All you Need](https://arxiv.org/abs/1706.03762) (which revolutionized both how we model neural networks for NLP, and how we give titles to our papers on the topic). This paper discusses many ingenious mechanisms, some of which we will discuss now.

Remember that in LSTMs we have the Forget Gate (if you have... ahem... *forgotten* about it, go back and read again!)? The Forget Gate is actually saying that information from the past $c_{t-1}$ should be weighted based on some criteria - and LSTM uses the output of an auxiliary MLP to find this weight. This means that the network memory has a relevance that is defined by the behavior of its surroundings.

## The base idea of attention

In the [Attention paper, Vaswani and their colleagues](https://arxiv.org/abs/1706.03762) observed that this is (at least in a metaphorical sense) similar to calculate a weighted average of the inputs, where weights represent the relevance of each input. Also, they observed that recurrent networks (even LSTMs) tend to give more relevance to words that are close by, even if they can potentially draw information from long distances - and this is not something desireable. Hence, they came with a clever solution.

Their solution starts with calculating the pairwise similarity between each item in the input (or: between each word embedding). For such, they use the inner product between each word, or simply:

$$
S = X X^T
$$

, where $X \in \mathbb{R}^{T \times D}$ contains $T$ word embeddings with dimension $D$.

The result $S \in \mathbb{R}^{T \times T}$ indicates the similarity between each pair of words in the sequence.

After that, $S$ is divided by $\sqrt{D}$ as a form of normalization, that is, this avoids finding larger values if the embedding dimension changes.

Finally, they normalize each line of $S$ using a softmax function, which transforms each line of $S$ into a probability distribution (that is, it sums to $1$).

At the end of this process, they have a weight matrix (they don't actually call it a weight matrix, but I am doing so):

$$
W = \text{softmax} (XX^T/\sqrt{D})
$$

At this point, each element $w_{t1,t2}$ of $W$ contains the weight given to token $t2$ at time $t1$. To apply these weights, they multiply $W$ by $X$, so that:

$$
Y = WX = \text{softmax} (XX^T/\sqrt{D})X.
$$

Thus, $Y \in \mathbb{R}^{T \times D}$ contains a transformation of the sequence (this is not stated, but I guess the name "transformer" comes from this idea) where each output is a weighted mean of the inputs. The weights can be seen as the "relevance" of each input, that is, points to which the system should pay *attention* - henceforth, *attention is all you need*.

## Query, key, value

The attention mechanism was observed to be similar to a search with a query in in a key-value database. In this type of database, we give relevance to values based on the similarity between a query and a key. Because of that, the output is actually calculated using three separated inputs, as in:

$$
Y = \text{softmax} (QK^T/\sqrt{D})V.
$$

An important step here is to allow three different inputs, instead of forcing $Q=K=V$. Thus, we have inputs $X_q, X_k, X_v$. Also, the paper proposed making linear projections of the inputs to obtain $Q,K,V$, that is:

$$
Q = X_qW_q^T\\
K = X_kW_k^T\\
V = X_vW_v^T,
$$

and this allows learning the weight matrices $W_q, W_k, W_v$.

## Positional encoding

It was also observed that the weighted mean in the attention mechanism [disconsiders the order of the words](https://arxiv.org/abs/1706.03762), which is something we were thriving for when using the LSTM. The proposed solution for this is to use an auxiliary sequence of tokens that represents the position of each word in the sequence. Then, each of these "position tokens" receives an embedding (this could be a trained embedding, although the transformer paper uses a [pre-defined cosine-sine embedding](https://arxiv.org/abs/1706.03762) - they both seem to work fine). This position encodings are added to the word embeddings and the result of this operation is, itself, propagated as the input of the network, as in:

<img src="figs/positional_encoding.png" />

## Multi-head attention and Pytorch implementation

It was observed that using multiple attention mechanisms in parallel can improve final results. Thus, the idea is to yield the same input to multiple attention mechanisms, and then cocatenate the results. This is somewhat ingenious, at it allows giving attention to different parts of the input for different reasons.

The pytorch [API for a multihead attention layer](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) is very simple:

    mhe = nn.MultiheadAttention(embed_dim, num_heads)

where `embed_dim` must be divisible by `num_heads`.

### Summarization for classification

You have probably noted that the output of a multi-head attention layer yields outputs for each time step in the input. We still need to summarize it. However, this time, we can use the *timewise mean* to summarize. This idea has been used in the [keras implementation of a multihead-attention classifier](https://keras.io/examples/nlp/text_classification_with_transformer/), and it draws from the fact that we have already considered the word positions in our representation when we used positional encoders.


Remeber that the multihead attention models sequence dependencies and yields:

$$
y = \text{softmax}(QK^T/\sqrt{D})V.
$$

The problem of this is that $QK^T$ is non-causal, that is, tokens get it touch with other, future tokens, thus prediction using this would be trivial again.

For such, we use a *mask*. It works like this:

$(QK^T)_{i,j}$ represents the attention given to word $i$ due to word $j$. Hence, we have two restrictions:

1. We cannot have word $i$ depend on word $j$ for any $j>i$,
1. We still need the $\text{softmax}$ operator because we want to make a weighted mean of the elements in $V$.

The solution for such is to make a matrix $M$ with the same shape as $QK^T$. The elements of this matrix are:

$$
m_{i,j} = 
\left\{
\begin{array}{rl}
0,& i \leq j \\
-\infty, & i > j  
\end{array}
\right.
$$

If we had three tokens in the sequence, we would get something like:

$$
M = 
\begin{bmatrix}
0 & -\infty & -\infty \\
0 & 0 & -\infty \\
0 & 0 & 0 \\
\end{bmatrix}
$$

Ok, then we add this matrix to $QK^T$ *before* the softmax operation. Hence, we get our masked output:

$$
y = \text{softmax}((M+QK^T)/\sqrt{D})V.
$$

See that after the $\text{softmax}$ normalization, the $-\infty$ terms go to zero, whereas the other elements still sum to one.

Lets implement this mask in Pytorch:

It seems to be a nice idea to use masked multi-head self-attention as a sequence model!

According to the [pytorch documentation](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html), the attention mask should be given as a parameter called `attn_mask` when we call the forward method in the `MultiHeadAttention` layer.

So we could have something like this:


In [3]:
import torch
import torch.nn as nn

def _make_causal_mask(seq_len):
    # The mask is True where we want to ignore attention and False elsewhere.
    mask = torch.ones(seq_len, seq_len)
    mask = torch.tril(mask)
    bool_mask = mask != 1
    return bool_mask
    
class MyLanguageModel( nn.Module ):
    def __init__(self, vocab_size, seq_len, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.position_embedding = nn.Embedding(seq_len, embedding_dim)
        self.sequence_model = nn.MultiheadAttention(embedding_dim, num_heads=1, batch_first=True)
        self.fc = nn.Linear(embedding_dim, vocab_size)
        self.seq_len = seq_len
        self.mask = _make_causal_mask(seq_len)

    def forward(self, x):
        e = self.embedding(x)
        p = self.position_embedding(torch.arange(self.seq_len).to(x.device)).unsqueeze(0)
        x = e + p
        x = self.sequence_model(x, x, x, attn_mask=self.mask, is_causal=True, need_weights=False)[0]
        x = self.fc(x)
        return x

lm = MyLanguageModel(
    vocab_size=3,
    seq_len=4,
    embedding_dim=5,
    )

x = torch.randint(0, 3, (2,4)) # Two sequences of length 4
y = lm(x)
print(y.shape)


torch.Size([2, 4, 3])


## Transformer architecture:

<img src="figs/transformer_original.png" />

<img src="figs/transformer_annotated.png" />

## Exercise (do it at home, it takes a while!)

1. Find some text you like
1. Use the `Dataset` and `Dataloader` from Pytorch to make it accessible by your transformer
1. Train your model!
1. Try to predict and reinsert some tokens/words in your model
1. Use the embeddings in a downstream task (e.g., classification)

# Further reading

We usually think that the last layer of the language model should be used in downstream tasks. However, [this paper](https://arxiv.org/abs/2502.02013) indicates that this is not always true! Check the paper - and see how it applies to the language model you have previously trained!